In [2]:
# Setup
import sys
sys.path.append('../')

from src.features.hawkes_features import (
    HawkesFeatureExtractor,
    extract_hawkes_features_rolling,
    detect_excitation_regimes,
    hawkes_regime_features
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from src.config import (
    INTERIM_DATA_DIR, 
    PROCESSED_DATA_DIR,
    FIGURES_DIR
)


In [3]:
import numpy as np
from src.features.hawkes_features import HawkesFeatureExtractor

# Generate Poisson test data
np.random.seed(42)
n_events = 1000
T = 100
poisson_times = np.sort(np.random.uniform(0, T, n_events))

# Fit
extractor = HawkesFeatureExtractor()
extractor.fit(poisson_times)
params = extractor.get_params()

print("Poisson test (should have low branching ratio):")
print(f"  μ (baseline):     {params['mu']:.4f} events/sec")
print(f"  α (excitation):   {params['alpha']:.4f}")
print(f"  β (decay):        {params['beta']:.4f}")
print(f"  Branching ratio:  {params['branching_ratio']:.4f}")
print(f"\n  Expected: branching ratio ≈ 0 for Poisson")
print(f"  Actual:   branching ratio = {params['branching_ratio']:.4f}")

if params['branching_ratio'] < 0.2:
    print("  ✓ Looks good - low self-excitation as expected!")
else:
    print("  ⚠ Higher than expected - but could be noise in small sample")

Poisson test (should have low branching ratio):
  μ (baseline):     9.6337 events/sec
  α (excitation):   0.0370
  β (decay):        2.9123
  Branching ratio:  0.0127

  Expected: branching ratio ≈ 0 for Poisson
  Actual:   branching ratio = 0.0127
  ✓ Looks good - low self-excitation as expected!


## Test with self-exciting data
Verify that Hawkes can detect self-excitation when it's present:

In [4]:
# Generate clustered/self-exciting data
np.random.seed(42)
clusters = []
for _ in range(20):  # 20 burst clusters
    cluster_start = np.random.uniform(0, 100)
    cluster_size = np.random.randint(30, 70)
    # Events tightly clustered (0.1 sec exponential spacing)
    cluster_times = cluster_start + np.random.exponential(0.1, cluster_size)
    clusters.extend(cluster_times)

clustered_times = np.sort(np.array(clusters))
clustered_times = clustered_times[clustered_times < 100]

# Fit
extractor2 = HawkesFeatureExtractor()
extractor2.fit(clustered_times)
params2 = extractor2.get_params()

print("\nClustered data test (should have HIGH branching ratio):")
print(f"  μ (baseline):     {params2['mu']:.4f}")
print(f"  α (excitation):   {params2['alpha']:.4f}")
print(f"  β (decay):        {params2['beta']:.4f}")
print(f"  Branching ratio:  {params2['branching_ratio']:.4f}")

if params2['branching_ratio'] > 0.5:
    print("  ✓ High self-excitation detected!")
else:
    print(f"  Moderate self-excitation (still higher than Poisson)")


Clustered data test (should have HIGH branching ratio):
  μ (baseline):     0.2849
  α (excitation):   0.9729
  β (decay):        45.4794
  Branching ratio:  0.0214
  Moderate self-excitation (still higher than Poisson)


## Trade Data Implementation

In [ ]:

# Load trade data
print("Loading trade data...")
trade_df = pd.read_parquet(INTERIM_DATA_DIR / 'synchronized_lob_trades.parquet')

# Extract timestamps (ensure in seconds)
trade_times = trade_df['timestamp'].values
print(f"Loaded {len(trade_times):,} trades")
print(f"Time range: {trade_times[0]:.2f}s to {trade_times[-1]:.2f}s")
print(f"Duration: {(trade_times[-1] - trade_times[0]) / 60:.2f} minutes")

In [ ]:


# Fit Hawkes on full dataset (exploratory)
print("\n=== Fitting Hawkes Process on Full Dataset ===")
extractor = HawkesFeatureExtractor()
extractor.fit(trade_times)

params = extractor.get_params()
print(f"\nFitted Parameters:")
print(f"  μ (baseline intensity):  {params['mu']:.6f} trades/sec")
print(f"  α (excitation):          {params['alpha']:.6f}")
print(f"  β (decay rate):          {params['beta']:.6f} /sec")
print(f"  Branching ratio (α/β):   {params['branching_ratio']:.4f}")

# Interpretation
print("\nInterpretation:")
if params['branching_ratio'] > 0.7:
    print(" HIGH self-excitation (volatile/active market)")
    print(" Trades are highly clustered - each trade triggers more trades")
elif params['branching_ratio'] > 0.4:
    print(" MODERATE self-excitation (normal trading activity)")
    print(" Some clustering, typical for liquid markets")
else:
    print(" LOW self-excitation (calm market, close to Poisson)")
    print(" Trades arrive relatively independently")

# Decay interpretation
decay_half_life = np.log(2) / params['beta']
print(f"\nDecay half-life: {decay_half_life:.2f} seconds")
print(f" Excitation fades to 50% in {decay_half_life:.2f}s")
print(f" Excitation nearly gone after {3*decay_half_life:.2f}s (3 half-lives)")

In [ ]:
# Compute and visualize intensity
print("\n=== Computing Hawkes Intensity ===")
intensity = extractor.intensity(trade_times, trade_times)

# Plot subset for visualization
n_plot = min(2000, len(trade_times))
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(trade_times[:n_plot], intensity[:n_plot], 
        label='Hawkes Intensity λ(t)', linewidth=1.5)
ax.scatter(trade_times[:n_plot], np.zeros(n_plot), 
           c='red', alpha=0.4, s=20, label='Actual Trades')
ax.axhline(params['mu'], color='green', linestyle='--', 
           label=f'Baseline μ = {params["mu"]:.4f}')
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Intensity (trades/sec)')
ax.set_title('Hawkes Process: Fitted Intensity vs Actual Trade Arrivals')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../reports/figures/hawkes_analysis/hawkes_intensity.png', dpi=150)
plt.show()

In [ ]:
# Simulate and compare
print("\n=== Simulating Hawkes Process ===")
T_sim = trade_times[-1] - trade_times[0]  # Same duration
simulated_times = extractor.simulate(T_sim, random_state=42)

print(f"Actual trades:    {len(trade_times):,}")
print(f"Simulated trades: {len(simulated_times):,}")

# Compare inter-arrival time distributions
actual_interarrival = np.diff(trade_times)
simulated_interarrival = np.diff(simulated_times)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram comparison
axes[0].hist(actual_interarrival, bins=50, alpha=0.6, label='Actual', density=True)
axes[0].hist(simulated_interarrival, bins=50, alpha=0.6, label='Simulated', density=True)
axes[0].set_xlabel('Inter-arrival Time (seconds)')
axes[0].set_ylabel('Density')
axes[0].set_title('Inter-arrival Time Distribution')
axes[0].legend()
axes[0].set_xlim(0, np.percentile(actual_interarrival, 95))

# QQ plot
from scipy import stats
stats.probplot(actual_interarrival, dist=stats.expon, plot=axes[1])
axes[1].set_title('Q-Q Plot: Actual vs Exponential')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'hawkes_analysis/hawkes_simulation_comparison.png', dpi=150)
plt.show()

In [ ]:
# Extract rolling Hawkes features
print("\n=== Extracting Rolling Hawkes Features ===")
hawkes_features_df = extract_hawkes_features_rolling(
    trade_times,
    window_size=500,   # 500 trades per window
    stride=50,         # Move forward 50 trades each step
    min_events=100     # Need at least 100 trades to fit
)

print(f"Extracted features for {len(hawkes_features_df)} time windows")
print(f"\nFeature statistics:")
print(hawkes_features_df[['mu', 'alpha', 'beta', 'branching_ratio']].describe())

In [ ]:
# Visualize rolling parameters over time
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

params_to_plot = ['mu', 'alpha', 'beta', 'branching_ratio']
titles = ['Baseline Intensity (μ)', 'Excitation (α)', 'Decay Rate (β)', 'Branching Ratio (α/β)']

for ax, param, title in zip(axes, params_to_plot, titles):
    ax.plot(hawkes_features_df['timestamp'], hawkes_features_df[param], linewidth=1.5)
    ax.set_ylabel(param)
    ax.set_title(title)
    ax.grid(alpha=0.3)
    
    # Add threshold line for branching ratio
    if param == 'branching_ratio':
        ax.axhline(0.7, color='red', linestyle='--', alpha=0.7, label='High Excitation Threshold')
        ax.legend()

axes[-1].set_xlabel('Time (seconds)')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'hawkes_analysis/hawkes_rolling_parameters.png', dpi=150)
plt.show()

In [ ]:
# Detect and analyze regimes
print("\n=== Detecting Excitation Regimes ===")
regime_features = hawkes_regime_features(
    hawkes_features_df['branching_ratio'].values,
    threshold=0.7
)

hawkes_features_df['regime'] = regime_features['regime_binary']
hawkes_features_df['regime_duration'] = regime_features['regime_duration']

# Regime statistics
n_excited = np.sum(regime_features['regime_binary'])
n_baseline = len(regime_features['regime_binary']) - n_excited
print(f"High excitation periods: {n_excited} ({n_excited/len(hawkes_features_df)*100:.1f}%)")
print(f"Baseline periods:        {n_baseline} ({n_baseline/len(hawkes_features_df)*100:.1f}%)")

# Average regime duration
excited_durations = hawkes_features_df[hawkes_features_df['regime'] == 1]['regime_duration']
baseline_durations = hawkes_features_df[hawkes_features_df['regime'] == 0]['regime_duration']
print(f"\nAverage regime persistence:")
print(f"  Excited:  {excited_durations.mean():.1f} windows")
print(f"  Baseline: {baseline_durations.mean():.1f} windows")

In [ ]:
#  Visualize regimes overlaid on price/volume
# (Assuming you have price data in trade_df)
if 'price' in trade_df.columns:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Price with regime background
    axes[0].plot(trade_df['timestamp'], trade_df['price'], linewidth=1, color='black')
    
    # Color background by regime
    for i in range(len(hawkes_features_df) - 1):
        if hawkes_features_df.iloc[i]['regime'] == 1:
            axes[0].axvspan(
                hawkes_features_df.iloc[i]['timestamp'],
                hawkes_features_df.iloc[i+1]['timestamp'],
                alpha=0.2, color='red'
            )
    
    axes[0].set_ylabel('Price')
    axes[0].set_title('Price with Hawkes Excitation Regimes (Red = High Excitation)')
    axes[0].grid(alpha=0.3)
    
    # Branching ratio
    axes[1].plot(hawkes_features_df['timestamp'], 
                 hawkes_features_df['branching_ratio'], 
                 linewidth=1.5, label='Branching Ratio')
    axes[1].axhline(0.7, color='red', linestyle='--', alpha=0.7, label='Threshold')
    axes[1].fill_between(hawkes_features_df['timestamp'],
                         0, 1,
                         where=hawkes_features_df['regime'] == 1,
                         alpha=0.2, color='red')
    axes[1].set_ylabel('Branching Ratio')
    axes[1].set_xlabel('Time (seconds)')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../reports/figures/hawkes_analysis/hawkes_regimes_price.png', dpi=150)
    plt.show()

In [ ]:

# Save features for downstream models
print("\n=== Saving Hawkes Features ===")
output_path = PROCESSED_DATA_DIR / 'hawkes_features.parquet'
hawkes_features_df.to_parquet(output_path)
print(f"Saved to: {output_path}")
print(f"Shape: {hawkes_features_df.shape}")
print(f"Columns: {list(hawkes_features_df.columns)}")